In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pathlib
import os
import cv2
import PIL
import PIL.Image
import time
import datetime

import matplotlib.pyplot as plt
print(tf.__version__)
import tensorboard
tensorboard.__version__

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
sys_details = tf.sysconfig.get_build_info()
cuda_version = sys_details["cuda_version"]
print(cuda_version)
cuda_version = sys_details["cudnn_version"]
print(cuda_version)
tf.config.list_physical_devices(device_type='GPU')

In [ ]:
data_dir = "/media/waqar/Data/PMS/road-only/two-classes/training"
data_dir = pathlib.Path(data_dir)
print("Training-Directory: ",data_dir)

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)


In [ ]:
one = list(data_dir.glob('1/*'))
print(one[0]);
print(len(one));

In [ ]:
### To test if the images are present in the dataset and opencv works
#temp = cv2.imread(str(one[0]));
#cv2.imshow("output",temp);
#PIL.Image.open(str(one[0]))

In [ ]:
## create a dataset using keras library function
batch_size = 45
img_height = 224
img_width = 224
IMG_SIZE = (img_height,img_width)
IMG_SHAPE = IMG_SIZE + (3,)
print(IMG_SHAPE)
train_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
class_names = train_dataset.class_names
print(class_names)

class_names_val= validation_dataset.class_names
print(class_names_val)


In [ ]:
class_names_val = validation_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in validation_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)



In [ ]:
#tf.keras.layers.RandomContrast
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomFlip('vertical'),
  tf.keras.layers.RandomRotation(0.1),
  tf.keras.layers.RandomRotation(0.3),
  tf.keras.layers.RandomZoom(0.1),
  tf.keras.layers.RandomContrast(0.1, 0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomContrast(0.1),
  tf.keras.layers.RandomContrast(0.09),
  tf.keras.layers.RandomContrast(0.08)
])


In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')


In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

In [ ]:

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)


In [ ]:
base_model.trainable = False



In [ ]:
base_model.summary()



In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
class_number = 2
prediction_layer = tf.keras.layers.Dense(class_number)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(224,224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.


In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
initial_epochs = 100

loss0, accuracy0 = model.evaluate(validation_dataset)
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))


In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    callbacks=tensorboard_callback)


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
###!rm -rf ./logs/ 

#%tensorboard --logdir ./logs

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


In [ ]:
!mkdir -p pms_model
model.save('pms_model/my_model_2_class_45_224_only_road',save_format='h5')


In [ ]:
model = tf.keras.models.load_model('pms_model/my_model_2_class_45_224_only_road')

# Check its architecture
model.summary()

In [ ]:
#Change the variables as per your need
test_data_dir = pathlib.Path('/media/waqar/Data/PMS/road-only/two-classes/test')
print(test_data_dir)
img_height = 224
img_width = 224
batch_size = 3
class_number = 2

test_dataset = tf.keras.utils.image_dataset_from_directory(
  test_data_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

In [ ]:
for image, label in test_dataset:
    prediction_batch = model(image)
    if(prediction_batch.shape[0] == label.shape[0]):
        print("true")
        print(tf.math.argmax(prediction_batch, axis=1).numpy())


In [ ]:
count=0
for i in range(1,4):
    print(i)
    count=0
    dir = pathlib.Path(data_dir,str(i))
    img_path = list(dir.glob('./*.jpg'))
    l = len(list(dir.glob('./*.jpg')))
    
    for x in range(0, l):
        #print(str(img_path[x]))
        img = tf.keras.utils.load_img(img_path[x], target_size=(img_height, img_width))
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch
     #   img_array = preprocess_input (img_array)
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
        #print(
        #    "This image most likely belongs to {} with a {:.2f} percent confidence."
        #    .format(class_names[np.argmax(score)], 100 * np.max(score))
        #)
        #print(int(class_names[np.argmax(score)]))
    
        if (int(class_names[np.argmax(score)])==i):
            count=count+1
  
        #else:
        #    print(int(class_names[np.argmax(score)]))
    print("No. of class: ",i)
    print("True positive: ",count)
    print("Total in class: ",l)

layer_names = [layer.name for layer in model.layers]
#layer_names
layer_outputs = [layer.output for layer in base_model.layers]
layer_outputs

feature_map_model = tf.keras.models.Model(inputs=base_model.input, outputs=layer_outputs)
#feature_map_model.summary()

img_path

img = tf.keras.utils.load_img(img_path, target_size=(img_height, img_width))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch
print(img_array.shape)
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
img_array = preprocess_input(img_array)
feature_maps = feature_map_model.predict(img_array)
for layer_name, feature_map in zip(layer_names, feature_maps):
    print(f"The shape of the {layer_name} is =======>> {feature_map.shape}")

for layer_name, feature_map in zip(layer_names, feature_maps):  
    if len(feature_map.shape) == 4:
        # Just do this for the convolutional layers, not the fully-connected layers
        n_features = feature_map.shape[-1]
        print(f"The number of features for {layer_name} is {n_features}")
        for i in range(n_features):
            feature_image = feature_map[0, :, :, i]
            plt.figure()
            plt.title(f"Feature map of {layer_name} #{i}")
            plt.imshow(feature_image)

